In [65]:
"""
Скрипт для обработки форм мониторинга кадровой потребности ОПК августовская форма
"""

'\nСкрипт для обработки форм мониторинга кадровой потребности ОПК августовская форма\n'

In [66]:
# -*- coding: UTF-8 -*-
import pandas as pd
import numpy as np
import tkinter
import sys
import os
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
from tkinter import ttk
import time
# pd.options.mode.chained_assignment = None  # default='warn'
import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None
import copy
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import re
import random
pd.options.display.width = 0

In [67]:
path_data_opk = 'data/Форма август'
path_to_end_folder = 'data'

In [68]:
error_df = pd.DataFrame(columns=['Название файла', 'Строка или колонка с ошибкой', 'Описание ошибки', ])

In [69]:
for file in os.listdir(path_data_opk):
    if not file.startswith('~$') and file.endswith('.xls'):
        name_file = file.split('.xls')[0]
        temp_error_df = pd.DataFrame(data=[[f'{name_file}', '',
                                            'Файл с расширением XLS (СТАРЫЙ ФОРМАТ EXCEL)!!! ДАННЫЕ ФАЙЛА НЕ ОБРАБОТАНЫ !!! ']],
                                     columns=['Название файла', 'Строка или колонка с ошибкой',
                                              'Описание ошибки'])
        error_df = pd.concat([error_df, temp_error_df], axis=0, ignore_index=True)
        continue
    if not file.startswith('~$') and file.endswith('.xlsx'):
        name_file = file.split('.xlsx')[0]
        print(name_file)
        # Проверяем наличие листов с названиями Форма 1 и Форма 2
        wb_1 = openpyxl.load_workbook(f'{path_data_opk}/{file}')
        if not {'Форма по мониторингу','Форма по принимаемым мерам'}.issubset(set(wb_1.sheetnames)):
            temp_error_df = pd.DataFrame(data=[[f'{name_file}', '',
                                                'Проверьте наличие листов с названием Форма по мониторингу и Форма по принимаемым мерам! Не должно быть пробелов в начале и в конце названия ДАННЫЕ ФАЙЛА НЕ ОБРАБОТАНЫ !!! ']],
                                         columns=['Название файла', 'Строка или колонка с ошибкой',
                                                  'Описание ошибки'])
            error_df = pd.concat([error_df, temp_error_df], axis=0, ignore_index=True)
            continue
    # считываем датафреймы
    df_mon = pd.read_excel(f'{path_data_opk}/{file}',sheet_name='Форма по мониторингу',dtype=str,skiprows=2) # лист с данными мониторинга
    df_action = pd.read_excel(f'{path_data_opk}/{file}',sheet_name='Форма по принимаемым мерам',dtype=str,skiprows=2) # лист с данными по принимаемым мерам
    
    # удаляем полностью пустые строки
    df_mon.dropna(thresh=20,inplace=True)
    df_action.dropna(thresh=7,inplace=True)
    
    # Проверяем совпадение ИНН.
    
        

Тестовая №1


In [73]:
df_mon

,гр. 1,гр.2,гр.3,гр.4,гр.5,гр.6,гр.7,гр.8,гр.9,гр.10,...,гр.16,гр.17,гр.18,гр.19,гр.20,гр.21,гр.22,гр.23,гр.24,гр.25
0,Республика Бурятия,0323053578,"АО ""У-У ППО""",Потребность имеется,Профессии рабочих,Авербандщик,2,29.02.08 Технология обработки алмазов,10,6,...,NaN,1,1,1,NaN,NaN,NaN,ВЕРНО,ВЕРНО,ВЕРНО
2,Республика Бурятия,0323053578,"АО ""У-У ППО""",Потребность имеется,Профессии рабочих,Сыросол,2,08.01.02 Монтажник трубопроводов,10,6,...,NaN,1,1,1,NaN,NaN,NaN,ВЕРНО,ВЕРНО,ВЕРНО
3,Республика Бурятия,0323053578,"АО ""У-У ППО""",Потребность имеется,"Должности служащих, руководителей",Техник службы эксплуатации,2,55.02.01 Театральная и аудиовизуальная техника...,10,6,...,NaN,1,1,1,NaN,NaN,NaN,ВЕРНО,ВЕРНО,ВЕРНО


In [ ]:
dct_mon = dict(zip(df_mon['гр.3'],df_mon['гр.2']))

In [72]:
error_df

,Название файла,Строка или колонка с ошибкой,Описание ошибки
